In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
df_restaurantes = pd.read_parquet(r'Datawarehouse\restaurantes.parquet')

In [3]:
df_restaurantes = df_restaurantes[df_restaurantes['nombre'] =="Subway"]
df_restaurantes

,id_restaurante,id_estado,nombre,categorias,cantidad_resenas,latitud,longitud,atributos,calificacion
1649,gi90NMmRhzl8vzM83UNOVA,9.0,Subway,"Food, Sandwiches, Restaurants, Fast Food",14,28.150110,-82.353997,"{'AcceptsInsurance': None, 'Accessibility': No...",3.000000
3209,wXBAAafoiZ4Nty2ajuBtfw,14.0,Subway,"Restaurants, Sandwiches, Fast Food",11,39.884360,-86.067905,"{'AcceptsInsurance': None, 'Accessibility': No...",2.500000
3262,pFtvNuoUppn9GYx0_ZsW5w,14.0,Subway,"Sandwiches, Restaurants, Fast Food",6,39.910207,-86.183382,"{'AcceptsInsurance': None, 'Accessibility': No...",2.500000
3724,CmDTNEVReN6CgE6ctlZhag,14.0,Subway,"Sandwiches, Salad, Fast Food, Restaurants",7,39.676701,-86.309883,"{'AcceptsInsurance': None, 'Accessibility': No...",3.000000
4609,31s1x27DnN2V-ptUWEdfTQ,14.0,Subway,"Sandwiches, Restaurants, Fast Food",7,39.912251,-86.261775,"{'AcceptsInsurance': None, 'Accessibility': No...",3.500000
...,...,...,...,...,...,...,...,...,...
211877,0x88179e0e2d642dab:0xb09b943bf4e08e40,22.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",117,42.220565,-85.594977,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.699219
211962,0x88ff5d3b16bb9a1f:0x957b1d97186d3b2f,40.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",8,33.982629,-80.487539,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.199219
211983,0x87b2117387ab144d:0x4b2675d40cc8855c,36.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",98,35.434847,-97.562387,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.900391
212004,0x87d268933b2e8637:0x7690da2187bb74d9,4.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",124,35.377148,-92.223073,"{'AcceptsInsurance': None, 'Accessibility': ['...",4.000000


In [4]:
df_restaurantes = df_restaurantes[df_restaurantes.nombre.str.contains('Subway')]
# df_restaurantes = df_restaurantes[df_restaurantes.nombre.str.lower().str.contains('subway')]

In [5]:
columnas = pd.DataFrame(pd.json_normalize(df_restaurantes.atributos).isna().sum(),columns=['nulos']).query("nulos < 6000").index.to_list()

In [6]:
columnas

['Accessibility',
 'Amenities',
 'Atmosphere',
 'Crowd',
 'Dining options',
 'Health & safety',
 'Highlights',
 'Offerings',
 'Payments',
 'Popular for',
 'Service options']

### 1. DF atributos

In [7]:
df_atributos = pd.json_normalize(df_restaurantes.atributos).reset_index()[columnas].fillna(np.ndarray([]))

In [8]:
for col in columnas:
    df_atributos[col] = df_atributos[col].apply(str)
    df_atributos[col] = df_atributos[col].apply(lambda c: "[]" if c == '0.0' else c)
    # df_atributos[col] = df_atributos[col].apply(json.loads)
    

In [9]:
df_atributos

,Accessibility,Amenities,Atmosphere,Crowd,Dining options,Health & safety,Highlights,Offerings,Payments,Popular for,Service options
0,[],[],[],[],[],[],[],[],[],[],[]
1,[],[],[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],[],[],[]
3,[],[],[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
8532,['Wheelchair accessible entrance'],['Good for kids'],['Casual'],['Tourists'],['Lunch'],['Mask required'],[],"['Comfort food' 'Healthy options' ""Kids' menu""...",['Debit cards' 'NFC mobile payments'],['Lunch' 'Dinner' 'Solo dining'],['Delivery' 'Takeout' 'Dine-in']
8533,['Wheelchair accessible entrance'],['Good for kids'],['Casual'],['Groups' 'Tourists'],['Lunch' 'Catering' 'Dessert'],[],[],"['Healthy options' ""Kids' menu"" 'Quick bite' '...",['Debit cards'],['Lunch' 'Dinner' 'Solo dining'],['Curbside pickup' 'Takeout' 'Dine-in' 'Delive...
8534,['Wheelchair accessible entrance'],['Good for kids'],['Casual'],['Groups' 'Tourists'],['Lunch' 'Catering' 'Dessert'],['Mask required'],[],"['Comfort food' 'Healthy options' ""Kids' menu""...",['Debit cards'],['Lunch' 'Dinner' 'Solo dining'],['Delivery' 'Takeout' 'Dine-in']
8535,['Wheelchair accessible entrance'],['Good for kids' 'High chairs'],['Casual'],['Groups' 'Tourists'],['Lunch' 'Catering' 'Dessert'],[],[],"['Coffee' 'Comfort food' 'Healthy options' ""Ki...",['Debit cards'],['Lunch' 'Dinner' 'Solo dining'],['Outdoor seating' 'Drive-through' 'Takeout' '...


#### 1.2 Accessibility

In [10]:
df_atributos['Accessibility'] = df_atributos['Accessibility'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

df_atributos['Accessibility']

0                                                      []
1                                                      []
2                                                      []
3                                                      []
4                                                      []
                              ...                        
8532                     [Wheelchair accessible entrance]
8533                     [Wheelchair accessible entrance]
8534                     [Wheelchair accessible entrance]
8535                     [Wheelchair accessible entrance]
8536    [Wheelchair accessible entrance, Wheelchair ac...
Name: Accessibility, Length: 8537, dtype: object

In [11]:
columnas_accesibilidad =  df_atributos['Accessibility'].explode().unique()[1:].tolist()


In [12]:
df_accesibilidad = pd.DataFrame([[False] * len(columnas_accesibilidad)] * df_atributos.shape[0], columns=columnas_accesibilidad)

In [13]:
df_accesibilidad

,Wheelchair accessible entrance,Wheelchair accessible parking lot,Wheelchair accessible restroom,Wheelchair accessible seating,Wheelchair accessible elevator,Wheelchair-accessible car park,Wheelchair-accessible entrance,Wheelchair-accessible seating,Wheelchair-accessible toilet,Wheelchair-accessible lift
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
8532,False,False,False,False,False,False,False,False,False,False
8533,False,False,False,False,False,False,False,False,False,False
8534,False,False,False,False,False,False,False,False,False,False
8535,False,False,False,False,False,False,False,False,False,False


In [14]:
for i, row in enumerate(df_atributos['Accessibility']):
    for j, val in enumerate(row):
        df_accesibilidad.loc[i, val] = True


In [15]:
df_restaurantes = df_restaurantes.reset_index().reset_index().drop(columns=['index']).rename({'level_0': 'index'}, axis=1, errors='ignore')
df_accesibilidad.reset_index().merge(df_restaurantes, on="index")


,index,Wheelchair accessible entrance,Wheelchair accessible parking lot,Wheelchair accessible restroom,Wheelchair accessible seating,Wheelchair accessible elevator,Wheelchair-accessible car park,Wheelchair-accessible entrance,Wheelchair-accessible seating,Wheelchair-accessible toilet,Wheelchair-accessible lift,id_restaurante,id_estado,nombre,categorias,cantidad_resenas,latitud,longitud,atributos,calificacion
0,0,False,False,False,False,False,False,False,False,False,False,gi90NMmRhzl8vzM83UNOVA,9.0,Subway,"Food, Sandwiches, Restaurants, Fast Food",14,28.150110,-82.353997,"{'AcceptsInsurance': None, 'Accessibility': No...",3.000000
1,1,False,False,False,False,False,False,False,False,False,False,wXBAAafoiZ4Nty2ajuBtfw,14.0,Subway,"Restaurants, Sandwiches, Fast Food",11,39.884360,-86.067905,"{'AcceptsInsurance': None, 'Accessibility': No...",2.500000
2,2,False,False,False,False,False,False,False,False,False,False,pFtvNuoUppn9GYx0_ZsW5w,14.0,Subway,"Sandwiches, Restaurants, Fast Food",6,39.910207,-86.183382,"{'AcceptsInsurance': None, 'Accessibility': No...",2.500000
3,3,False,False,False,False,False,False,False,False,False,False,CmDTNEVReN6CgE6ctlZhag,14.0,Subway,"Sandwiches, Salad, Fast Food, Restaurants",7,39.676701,-86.309883,"{'AcceptsInsurance': None, 'Accessibility': No...",3.000000
4,4,False,False,False,False,False,False,False,False,False,False,31s1x27DnN2V-ptUWEdfTQ,14.0,Subway,"Sandwiches, Restaurants, Fast Food",7,39.912251,-86.261775,"{'AcceptsInsurance': None, 'Accessibility': No...",3.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8532,8532,True,False,False,False,False,False,False,False,False,False,0x88179e0e2d642dab:0xb09b943bf4e08e40,22.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",117,42.220565,-85.594977,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.699219
8533,8533,True,False,False,False,False,False,False,False,False,False,0x88ff5d3b16bb9a1f:0x957b1d97186d3b2f,40.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",8,33.982629,-80.487539,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.199219
8534,8534,True,False,False,False,False,False,False,False,False,False,0x87b2117387ab144d:0x4b2675d40cc8855c,36.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",98,35.434847,-97.562387,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.900391
8535,8535,True,False,False,False,False,False,False,False,False,False,0x87d268933b2e8637:0x7690da2187bb74d9,4.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",124,35.377148,-92.223073,"{'AcceptsInsurance': None, 'Accessibility': ['...",4.000000


In [16]:
df_accesibilidad.columns = ["access_" + c for c in df_accesibilidad.columns]
union = df_accesibilidad.reset_index().merge(df_restaurantes, on="index")

In [17]:
union

,index,access_Wheelchair accessible entrance,access_Wheelchair accessible parking lot,access_Wheelchair accessible restroom,access_Wheelchair accessible seating,access_Wheelchair accessible elevator,access_Wheelchair-accessible car park,access_Wheelchair-accessible entrance,access_Wheelchair-accessible seating,access_Wheelchair-accessible toilet,access_Wheelchair-accessible lift,id_restaurante,id_estado,nombre,categorias,cantidad_resenas,latitud,longitud,atributos,calificacion
0,0,False,False,False,False,False,False,False,False,False,False,gi90NMmRhzl8vzM83UNOVA,9.0,Subway,"Food, Sandwiches, Restaurants, Fast Food",14,28.150110,-82.353997,"{'AcceptsInsurance': None, 'Accessibility': No...",3.000000
1,1,False,False,False,False,False,False,False,False,False,False,wXBAAafoiZ4Nty2ajuBtfw,14.0,Subway,"Restaurants, Sandwiches, Fast Food",11,39.884360,-86.067905,"{'AcceptsInsurance': None, 'Accessibility': No...",2.500000
2,2,False,False,False,False,False,False,False,False,False,False,pFtvNuoUppn9GYx0_ZsW5w,14.0,Subway,"Sandwiches, Restaurants, Fast Food",6,39.910207,-86.183382,"{'AcceptsInsurance': None, 'Accessibility': No...",2.500000
3,3,False,False,False,False,False,False,False,False,False,False,CmDTNEVReN6CgE6ctlZhag,14.0,Subway,"Sandwiches, Salad, Fast Food, Restaurants",7,39.676701,-86.309883,"{'AcceptsInsurance': None, 'Accessibility': No...",3.000000
4,4,False,False,False,False,False,False,False,False,False,False,31s1x27DnN2V-ptUWEdfTQ,14.0,Subway,"Sandwiches, Restaurants, Fast Food",7,39.912251,-86.261775,"{'AcceptsInsurance': None, 'Accessibility': No...",3.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8532,8532,True,False,False,False,False,False,False,False,False,False,0x88179e0e2d642dab:0xb09b943bf4e08e40,22.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",117,42.220565,-85.594977,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.699219
8533,8533,True,False,False,False,False,False,False,False,False,False,0x88ff5d3b16bb9a1f:0x957b1d97186d3b2f,40.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",8,33.982629,-80.487539,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.199219
8534,8534,True,False,False,False,False,False,False,False,False,False,0x87b2117387ab144d:0x4b2675d40cc8855c,36.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",98,35.434847,-97.562387,"{'AcceptsInsurance': None, 'Accessibility': ['...",3.900391
8535,8535,True,False,False,False,False,False,False,False,False,False,0x87d268933b2e8637:0x7690da2187bb74d9,4.0,Subway,"['Sandwich shop', 'Caterer', 'Fast food restau...",124,35.377148,-92.223073,"{'AcceptsInsurance': None, 'Accessibility': ['...",4.000000


In [18]:
df_atributos['Accessibility'].iloc[8536]

['Wheelchair accessible entrance',
 'Wheelchair accessible parking lot',
 'Wheelchair accessible restroom']

#### 1.3 DF Amenities

In [19]:
df_atributos['Amenities'] = df_atributos['Amenities'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_amenities = df_atributos['Amenities'].explode().unique()[1:].tolist()
df_amenities = pd.DataFrame([[False] * len(columnas_amenities)] * df_atributos.shape[0], columns=columnas_amenities)

for i, row in enumerate(df_atributos['Amenities']):
    for j, val in enumerate(row):
        df_amenities.loc[i, val] = True

df_amenities.columns = ["amen_" + c for c in df_amenities.columns]
union = df_amenities.reset_index().merge(union, on="index")

#### 1.4 Atmosphere

In [20]:
df_atributos['Atmosphere'] = df_atributos['Atmosphere'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_atmosphere = df_atributos['Atmosphere'].explode().unique()[1:].tolist()

df_atmosphere = pd.DataFrame([[False] * len(columnas_atmosphere)] * df_atributos.shape[0], columns=columnas_atmosphere)

for i, row in enumerate(df_atributos['Atmosphere']):
    for j, val in enumerate(row):
        df_atmosphere.loc[i, val] = True

df_atmosphere.columns = ["atmos_" + c for c in df_atmosphere.columns]
union = df_atmosphere.reset_index().merge(union, on="index")

#### 1.5 Crowd

In [21]:
df_atributos['Crowd'] = df_atributos['Crowd'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_crowd = df_atributos['Crowd'].explode().unique()[1:].tolist()

df_crowd = pd.DataFrame([[False] * len(columnas_crowd)] * df_atributos.shape[0], columns=columnas_crowd)

for i, row in enumerate(df_atributos['Crowd']):
    for j, val in enumerate(row):
        df_crowd.loc[i, val] = True

df_crowd.columns = ["crowd_" + c for c in df_crowd.columns]
union = df_crowd.reset_index().merge(union, on="index")       

#### 1.6 Dinning Options

In [22]:
df_atributos['Dining options'] = df_atributos['Dining options'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_dining = df_atributos['Dining options'].explode().unique()[1:].tolist()
df_dining = pd.DataFrame([[False] * len(columnas_dining)] * df_atributos.shape[0], columns=columnas_dining)

for i, row in enumerate(df_atributos['Dining options']):
    for j, val in enumerate(row):
        df_dining.loc[i, val] = True

df_dining.columns = ["dining_" + c for c in df_dining.columns]
union = df_dining.reset_index().merge(union, on="index")

#### 1.7 Health & safety

In [23]:
df_atributos['Health & safety'] = df_atributos['Health & safety'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_health = df_atributos['Health & safety'].explode().unique()[1:].tolist()

df_health = pd.DataFrame([[False] * len(columnas_health)] * df_atributos.shape[0], columns=columnas_health)

for i, row in enumerate(df_atributos['Health & safety']):
    for j, val in enumerate(row):
        df_health.loc[i, val] = True

df_health.columns = ["health_" + c for c in df_health.columns]
union = df_health.reset_index().merge(union, on="index")        


#### 1.8 Highlights

In [24]:
df_atributos['Highlights'] = df_atributos['Highlights'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_high = df_atributos['Highlights'].explode().unique()[1:].tolist()

df_high = pd.DataFrame([[False] * len(columnas_high)] * df_atributos.shape[0], columns=columnas_high)

for i, row in enumerate(df_atributos['Highlights']):
    for j, val in enumerate(row):
        df_high.loc[i, val] = True

df_high.columns = ["high_" + c for c in df_high.columns]
union = df_high.reset_index().merge(union, on="index")

#### 1.9 Offerings

In [25]:
df_atributos['Offerings'] = df_atributos['Offerings'].apply(lambda r: r.replace("Kids'", "Kids"))

df_atributos['Offerings'] = df_atributos['Offerings'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_offerings = df_atributos['Offerings'].explode().unique()[1:].tolist()

df_offering = pd.DataFrame([[False] * len(columnas_offerings)] * df_atributos.shape[0], columns=columnas_offerings)

for i, row in enumerate(df_atributos['Offerings']):
    for j, val in enumerate(row):
        df_offering.loc[i, val] = True

df_offering.columns = ["offer_" + c for c in df_offering.columns]
union = df_offering.reset_index().merge(union, on="index")

#### 1.10 Payments

In [26]:
df_atributos['Payments'] = df_atributos['Payments'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_payments = df_atributos['Payments'].explode().unique()[1:].tolist()

df_payments = pd.DataFrame([[False] * len(columnas_payments)] * df_atributos.shape[0], columns=columnas_payments)

for i, row in enumerate(df_atributos['Payments']):
    for j, val in enumerate(row):
        df_payments.loc[i, val] = True

df_payments.columns = ["pay_" + c for c in df_payments.columns]
union = df_payments.reset_index().merge(union, on="index")

#### 1.11 Popular for

In [27]:
df_atributos['Popular for'] = df_atributos['Popular for'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_popular = df_atributos['Popular for'].explode().unique()[1:].tolist()

df_popular = pd.DataFrame([[False] * len(columnas_popular)] * df_atributos.shape[0], columns=columnas_popular)

for i, row in enumerate(df_atributos['Popular for']):
    for j, val in enumerate(row):
        df_popular.loc[i, val] = True

df_popular.columns = ["popular_" + c for c in df_popular.columns]
union = df_popular.reset_index().merge(union, on="index")



#### 1.12 Service Options

In [28]:
df_atributos['Service options'] = df_atributos['Service options'].apply(lambda s: re.sub(r"'", "\"", s)).apply(lambda s: re.sub(r'"\s*"','","', s)).apply(json.loads)

columnas_service = df_atributos['Service options'].explode().unique()[1:].tolist()

df_service = pd.DataFrame([[False] * len(columnas_service)] * df_atributos.shape[0], columns=columnas_service)

for i, row in enumerate(df_atributos['Service options']):
    for j, val in enumerate(row):
        df_service.loc[i, val] = True

df_service.columns = ["service_" + c for c in df_service.columns]

union = df_service.reset_index().merge(union, on="index")

In [29]:
union.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8537 entries, 0 to 8536
Data columns (total 82 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   index                                                       8537 non-null   int64  
 1   service_Curbside pickup                                     8537 non-null   bool   
 2   service_Delivery                                            8537 non-null   bool   
 3   service_Takeout                                             8537 non-null   bool   
 4   service_Dine-in                                             8537 non-null   bool   
 5   service_Drive-through                                       8537 non-null   bool   
 6   service_No-contact delivery                                 8537 non-null   bool   
 7   service_Outdoor seating                                     8537 non-null   bool   
 8 

# 2. Machine Learning

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score



In [166]:
df_restaurantes.columns

dropear = ["index", "id_restaurante","categorias", "atributos","nombre","id_estado", "cantidad_resenas"]

df = union.drop(columns=dropear)

df['calificacion'] = df['calificacion'].apply(lambda x: 1 if x >= 3.7 else 0)

In [167]:
df = pd.concat([df.query("calificacion == 1").sample(3350),df.query("calificacion == 0")])

In [168]:
df['calificacion'].value_counts()

calificacion
1    3350
0    3350
Name: count, dtype: int64

In [169]:
X = df.iloc[:, :-1]  # Todas las columnas excepto la última
y = df.iloc[:, -1]   # La última columna

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo de regresión
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)
y_pred_reg = reg_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_reg)
print(f'Mean Squared Error (Regresión): {mse}')

# Modelo de clasificación
clf_model = RandomForestClassifier(random_state=42)
clf_model.fit(X_train, y_train)
y_pred_clf = clf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_clf)
print(f'Accuracy (Clasificación): {accuracy}')
random_forest_accuracy = accuracy

Mean Squared Error (Regresión): 0.22326940111737273
Accuracy (Clasificación): 0.6044776119402985


In [170]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Crear y entrenar el modelo de regresión logística
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_log_reg = log_reg_model.predict(X_test)

# Evaluar el modelo
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print(f'Accuracy (Regresión Logística): {accuracy_log_reg}')
linear_reg_accuracy = accuracy_log_reg

Accuracy (Regresión Logística): 0.6253731343283582


g:\cursos_2023\HENRY\DATA07\PF\PF_DATAPT07\.venv10\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [171]:
    import xgboost as xgb

    # Convertir los datos al formato DMatrix para XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Definir los parámetros del modelo
    params = {
        'objective': 'multi:softmax',  # Para clasificación multiclase
        'num_class': 2,  # Número de clases en la variable de salida
        'eval_metric': 'merror',  # Métrica de evaluación
        'learning_rate': 0.1,
        'max_depth': 5,
    }

    # Entrenar el modelo XGBoost
    num_rounds = 100
    xgb_model = xgb.train(params, dtrain, num_rounds)

    # Hacer predicciones
    y_pred_xgb = xgb_model.predict(dtest)

    # Evaluar el modelo
    accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
    print(f'Accuracy (XGBoost): {accuracy_xgb}')

    xgboost_accuracy = accuracy_xgb


Accuracy (XGBoost): 0.6223880597014926


In [172]:
print(f"Columnas dropear {dropear}")
print(f"random forest: {round(random_forest_accuracy*100, 3)}")
print(f"linear reg   : {round(linear_reg_accuracy*100, 3)}" )
print(f"xgboost      : {round(xgboost_accuracy*100, 3)}"  )

Columnas dropear ['index', 'id_restaurante', 'categorias', 'atributos', 'nombre', 'id_estado', 'cantidad_resenas']
random forest: 60.448
linear reg   : 62.537
xgboost      : 62.239


In [173]:
# %pip install xgboost

In [174]:
# %pip install torch

In [175]:
X_train

,service_Curbside pickup,service_Delivery,service_Takeout,service_Dine-in,service_Drive-through,service_No-contact delivery,service_Outdoor seating,service_Takeaway,popular_Lunch,popular_Dinner,...,access_Wheelchair accessible restroom,access_Wheelchair accessible seating,access_Wheelchair accessible elevator,access_Wheelchair-accessible car park,access_Wheelchair-accessible entrance,access_Wheelchair-accessible seating,access_Wheelchair-accessible toilet,access_Wheelchair-accessible lift,latitud,longitud
4412,True,True,True,True,False,False,False,False,True,True,...,True,True,False,False,False,False,False,False,29.942730,-95.714803
3355,True,True,True,True,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,38.360076,-77.520231
8168,True,True,True,True,False,False,False,False,True,True,...,True,True,False,False,False,False,False,False,28.867137,-81.956460
7235,False,True,True,True,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,41.274219,-80.772305
3076,False,True,True,True,False,False,False,False,True,True,...,True,True,True,False,False,False,False,False,39.952458,-82.979524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,33.045299,-97.004670
4011,True,True,True,True,True,False,False,False,True,True,...,False,False,False,False,False,False,False,False,42.912009,-85.647407
4108,False,True,True,True,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,44.852671,-87.358838
4557,True,True,True,True,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,28.627299,-81.364370


### 3.Resto